## Web Scraping

In [18]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import pandas as pd
from easynmt import EasyNMT

In [34]:
urls = ["https://news.google.com/search?q=kendaraan%20listrik&hl=en-ID&gl=ID&ceid=ID%3Aen","https://news.google.com/search?q=mobil%20listrik&hl=en-ID&gl=ID&ceid=ID%3Aen"]

titles = []

for i in urls:
    soup = BeautifulSoup(requests.get(i).text, 'html.parser')

    # Find all anchor tags with the specified class
    containers = soup.find_all('a', class_=lambda x: x and x.startswith('DY5T1d RZIKme'))

    # Iterate over the ResultSet and print the text content of each anchor tag
    for container in containers:
        text_content = container.text
        titles.append(text_content)
        print(text_content)

Kemenko Marves dorong percepatan peralihan kendaraan listrik
Mengulas Langkah Indonesia Jadi Produsen Baterai dan Kendaraan Listrik
RI Percepat Pengembangan Ekosistem Kendaraan Listrik, Begini Caranya
Pemkot Surabaya Godok Tarif Angkutan Umum Listrik - harian.disway.id
Isuzu Masih Belum Yakin Bawa Truk Listrik ke Indonesia
Pembiayaan kendaraan listrik bisa capai Rp170 miliar pada 2023
Presiden Syaratkan IKN 100 Persen Pakai Kendaraan Listrik, PLN: Kami Siapkan Infrastrur | Republika Online
IKN Pakai 100 Persen Kendaraan Listrik, 19 SPBU Listrik Sudah Terpasang pada 2024
PLN Kembangkan Layanan One Stop Solution EVDS untuk Mudahkan Pengguna Kendaraan Listrik
Imam Budi Hartono Ajak Pejabat Pemkot Depok dan Masyarakat Gunakan Kendaraan Listrik
Pembiayaan Kendaraan Listrik pada 2023 Diprediksi Tembus Rp 170 Miliar
Adira Finance Sambut Baik Peningkatan Subsidi Kendaraan Listrik, Begini Catatannya
Pembiayaan Kendaraan Listrik Diprediksi Mencapai Rp 170 Miliar di 2023
Sokonindo Automobile Doro

In [38]:
len(titles)

211

In [40]:
# Make a dataframe based on the container
df = pd.DataFrame(titles)

# Rename the first column to title
df.rename(columns={0:'title'}, inplace=True)

In [41]:
print(df)

                                                 title
0    Kemenko Marves dorong percepatan peralihan ken...
1    Mengulas Langkah Indonesia Jadi Produsen Bater...
2    RI Percepat Pengembangan Ekosistem Kendaraan L...
3    Pemkot Surabaya Godok Tarif Angkutan Umum List...
4    Isuzu Masih Belum Yakin Bawa Truk Listrik ke I...
..                                                 ...
206  Program Mobil Listrik Amerika Terancam Mati DI...
207  Terungkap! Harga yang Mahal Bikin Mobil Listri...
208  RI Tak Punya UU Mobil Listrik, ESDM Ungkap Ala...
209  INFOGRAFIS: 10 Mobil Listrik Populer Januari -...
210  Anies Kritik Subsidi Mobil Listrik, Harganya M...

[211 rows x 1 columns]


## Machine Translation

In [19]:
#Load the model
model = EasyNMT('mbart50_m2en')

1.47kB [00:00, 1.47MB/s]                 
e:\VM\miniconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 1.51k/1.51k [00:00<00:00, 756kB/s]
e:\VM\miniconda3\envs\nlp\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mryan\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to r

In [42]:
# Translate from indonesia to english by iterating the first column of the dataframe, make a new column for the translated text from indonesian to english
df['translated'] = df['title'].apply(lambda x: model.translate(x, source_lang='id', target_lang='en'))

In [43]:
# Print the dataframe as a table
df

,title,translated
0,Kemenko Marves dorong percepatan peralihan ken...,Kemenko Marves pushed the switch to electric v...
1,Mengulas Langkah Indonesia Jadi Produsen Bater...,"Step by step, Indonesia became a battery and e..."
2,RI Percepat Pengembangan Ekosistem Kendaraan L...,RI Speeds Up Development of a Power Stewardshi...
3,Pemkot Surabaya Godok Tarif Angkutan Umum List...,The shutdown of Surabaya's public transportati...
4,Isuzu Masih Belum Yakin Bawa Truk Listrik ke I...,Isuzu Still Hasn't Bought Electricity to Indon...
...,...,...
206,Program Mobil Listrik Amerika Terancam Mati DI...,America's electric car program is under threat...
207,Terungkap! Harga yang Mahal Bikin Mobil Listri...,"Revealed! There's a huge under-priced, under-p..."
208,"RI Tak Punya UU Mobil Listrik, ESDM Ungkap Ala...","RI Won't Pass the Electric Car Law, ESDM Expla..."
209,INFOGRAFIS: 10 Mobil Listrik Populer Januari -...,INFOGRAPHY: 10 Most Popular Electric Cars Janu...


In [45]:
# Convert the dataframe to csv
df.to_csv('translated.csv')

## TOPIC MODELING

In [46]:
from bertopic import BERTopic
docs = list(df['translated'].values)
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(docs)

In [47]:
topic_model.get_topic_info().head(7).set_index('Topic')[
   ['Count', 'Name', 'Representation']]

,Count,Name,Representation
Topic,,,
-1,61,-1_the_to_electric_is,"[the, to, electric, is, of, in, electricity, c..."
0,72,0_electric_car_the_cars,"[electric, car, the, cars, to, chinas, of, in,..."
1,29,1_power_president_indonesia_of,"[power, president, indonesia, of, joko, for, t..."
2,29,2_indonesia_electric_in_cars,"[indonesia, electric, in, cars, car, vehicle, ..."
3,20,3_the_imam_founder_khomeini,"[the, imam, founder, khomeini, foundation, of,..."


In [48]:
topic_model.visualize_barchart(top_n_topics = 16, n_words = 10)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.03687032638765517, 0.039351101049000185,
                    0.039838322758757885, 0.04024118304789192, 0.04461445115495399,
                    0.044728858001259024, 0.06598494043053425, 0.08632119396554039,
                    0.10784425826318429, 0.1269491130455883],
              'xaxis': 'x',
              'y': [is  , ri  , in  , of  , chinas  , to  , cars  , the  , car  ,
                    electric  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.05198367277894517, 0.05766894629320731, 0.05808174437155498,
                    0.06029555190500916, 0.06032334809404467, 0.0698068994850734,
                    0.07387796822982196, 0.09649421623897304, 0.10561689276169275,
                    0.10740368263495693],
              'xaxis': 'x2',
              'y': [indonesiagoid  , widodo  , electricity  , to  , for  , joko  ,
                    of  , indonesia  , president  , power  ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.05821205545729791, 0.06489841590109971, 0.06546177894322679,
                    0.07337367537117204, 0.07936512991919048, 0.09708261935274937,
                    0.10052373464359723, 0.14514928488578324, 0.15237470054761162,
                    0.17242178200949343],
              'xaxis': 'x3',
              'y': [is  , the  , without  , indonesian  , vehicle  , car  , cars
                    , in  , electric  , indonesia  ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.048608109611481196, 0.048608109611481196,
                    0.07345729662768685, 0.07532206314659318, 0.16328164879129478,
                    0.1814299651727613, 0.1814299651727613, 0.18889019132833762,
                    0.18889019132833762, 0.22681334128700664],
              'xaxis': 'x4',
              'y': [more  , ling  , electrical  , power  , of  , foundation  ,
                    khomeini  , founder  , imam  , the  ],
              'yaxis': 'y4'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 0',
                                'x': 0.0875,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 1',
                                'x': 0.36250000000000004,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 2',
                                'x': 0.6375000000000001,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 3',
                                'x': 0.9125,
                                'xanchor': 'cent